<h1>Stacked Auto Encoders Training 7-13-2016 With Noise</h1>

Assuming CNN-H1, and the 6 CNNs for the patches have been trained, we can use the weights to get activations for their fc6 layer and train 3 layers of our SAE individually.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu1')

import os
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Process
from scipy.ndimage.filters import gaussian_filter

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers

# from keras.utils import np_utils

np.random.seed(123456)

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using gpu device 1: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)
Using Theano backend.


In [3]:
cnn_h1_weight_file = 'CNN-H1_epoch-20_activations'
cnn_p1_weight_file = 'CNN-P1_epoch-20_activations'
cnn_p2_weight_file = 'CNN-P2_epoch-20_activations'
cnn_p3_weight_file = 'CNN-P3_epoch-20_activations'
cnn_p4_weight_file = 'CNN-P4_epoch-20_activations'
cnn_p5_weight_file = 'CNN-P5_epoch-20_activations'
cnn_p6_weight_file = 'CNN-P6_epoch-20_activations'
y_train_file = 'y_train_activations'

layer_1_size = 2048
layer_2_size = 1024
layer_3_size = 512

validation_split = .15


In [4]:
def load_activations(file_name):
    return np.load('models/' + file_name + '.npy')

def save_activations(file_name, activations):
    return np.save('models/' + file_name + '.npy', activations)

In [5]:
cnn_h1_activations = load_activations(cnn_h1_weight_file)
cnn_p1_activations = load_activations(cnn_p1_weight_file)
cnn_p2_activations = load_activations(cnn_p2_weight_file)
cnn_p3_activations = load_activations(cnn_p3_weight_file)
cnn_p4_activations = load_activations(cnn_p4_weight_file)
cnn_p5_activations = load_activations(cnn_p5_weight_file)
cnn_p6_activations = load_activations(cnn_p6_weight_file)
y_train = load_activations(y_train_file)

In [6]:
def fix_activations(_activations):
    return np.array([activation for activations in _activations for activation in activations])

In [7]:
cnn_h1_activations = fix_activations(cnn_h1_activations)
cnn_p1_activations = fix_activations(cnn_p1_activations)
cnn_p2_activations = fix_activations(cnn_p2_activations)
cnn_p3_activations = fix_activations(cnn_p3_activations)
cnn_p4_activations = fix_activations(cnn_p4_activations)
cnn_p5_activations = fix_activations(cnn_p5_activations)
cnn_p6_activations = fix_activations(cnn_p6_activations)

In [8]:
cnn_h1_activations.shape

(26668, 512)

In [9]:
activations = np.concatenate((
        cnn_h1_activations, 
        cnn_p1_activations,
        cnn_p2_activations,
        cnn_p3_activations,
        cnn_p4_activations,
        cnn_p5_activations,
        cnn_p6_activations), axis=1)

In [10]:
activations.shape

(26668, 3584)

In [11]:
y_train.shape

(26668,)

<h2>Random Noise</h2>

In [12]:
def random_noise(activations, random_threshold, gaussian_scale):
    return np.array([np.array([value + np.random.normal(scale=gaussian_scale) 
                      if np.random.rand() < random_threshold else value
                      for value in activation]) for activation in activations])       

In [ ]:
activations_with_noise_10_1 = random_noise(activations, .1, 1)
save_activations('activations_with_noise_10_1', activations_with_noise_10_1)

In [65]:
activations_with_noise_05_1 = random_noise(activations, .05, 1)
save_activations('activations_with_noise_05_1', activations_with_noise_05_1)

KeyboardInterrupt: 

In [20]:
activations_with_noise_20_1 = random_noise(activations, .2, 1)
save_activations('activations_with_noise_20_1', activations_with_noise_20_1)

In [28]:
activations_with_noise_30_1 = random_noise(activations, .3, 1)
save_activations('activations_with_noise_30_1', activations_with_noise_30_1)

<h2>Auto Encoder</h2>

In [29]:
def get_encoder(X_train, X_test, encoding_dim, nb_epoch=50, regularizer=10e-5):
    assert X_train.shape[1] == X_test.shape[1]
    shape = X_train.shape[1]

    input_img = Input(shape=(shape,))
    encoded = Dense(
        encoding_dim, 
        activation='relu', 
        activity_regularizer=regularizers.activity_l1(regularizer)
    )(input_img)
    decoded = Dense(shape, activation='sigmoid')(encoded)

    autoencoder = Model(input=input_img, output=decoded)
    encoder = Model(input=input_img, output=encoded)

    # sgd = SGD(lr=.01, decay=99.9) #lr=0.01->0.00001
    autoencoder.compile(
        optimizer='adadelta', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])
    
    autoencoder.fit(X_train, X_train,
                nb_epoch=nb_epoch,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))
    return autoencoder, encoder
    

In [34]:
activations_with_noise = load_activations('activations_with_noise_30_1')

x_train = activations_with_noise[:int(activations_with_noise.shape[0] * (1 - validation_split))]
x_test = activations_with_noise[int(activations_with_noise.shape[0] * (1 - validation_split)):]
X_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
X_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [35]:
autoencoder1, encoder1 = get_encoder(X_train, X_test, 2048, 30, 10e-3)

Train on 22667 samples, validate on 4001 samples
Epoch 1/30
22667/22667 [==============================] - 4s - loss: -57973.8529 - acc: 0.0414 - val_loss: -65449.6027 - val_acc: 0.0757
Epoch 2/30
22667/22667 [==============================] - 4s - loss: -67489.7975 - acc: 0.1147 - val_loss: -68494.9747 - val_acc: 0.1427
Epoch 3/30
22667/22667 [==============================] - 4s - loss: -69783.4737 - acc: 0.1586 - val_loss: -69821.3545 - val_acc: 0.1617
Epoch 4/30
22667/22667 [==============================] - 4s - loss: -71011.6880 - acc: 0.1919 - val_loss: -71128.5413 - val_acc: 0.2002
Epoch 5/30
22667/22667 [==============================] - 4s - loss: -71876.4442 - acc: 0.2160 - val_loss: -71661.0824 - val_acc: 0.2219
Epoch 6/30
22667/22667 [==============================] - 4s - loss: -72516.0055 - acc: 0.2373 - val_loss: -71997.6701 - val_acc: 0.2362
Epoch 7/30
22667/22667 [==============================] - 4s - loss: -72986.8669 - acc: 0.2503 - val_loss: -72368.1699 - val_acc:

In [36]:
X_train = encoder1.predict(X_train)
X_test = encoder1.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 2048)
(4001, 2048)


In [37]:
autoencoder2, encoder2 = get_encoder(X_train, X_test, 1024, 50)

Train on 22667 samples, validate on 4001 samples
Epoch 1/50
22667/22667 [==============================] - 2s - loss: 90460.4151 - acc: 5.2940e-04 - val_loss: 89618.6260 - val_acc: 9.9975e-04
Epoch 2/50
22667/22667 [==============================] - 2s - loss: 89522.8048 - acc: 0.0012 - val_loss: 88951.2154 - val_acc: 0.0042
Epoch 3/50
22667/22667 [==============================] - 2s - loss: 88566.5401 - acc: 0.0196 - val_loss: 87696.9567 - val_acc: 0.0835
Epoch 4/50
22667/22667 [==============================] - 2s - loss: 87095.2562 - acc: 0.1074 - val_loss: 86337.9030 - val_acc: 0.1620
Epoch 5/50
22667/22667 [==============================] - 2s - loss: 85858.9567 - acc: 0.1735 - val_loss: 85405.2249 - val_acc: 0.2044
Epoch 6/50
22667/22667 [==============================] - 2s - loss: 84921.3315 - acc: 0.2313 - val_loss: 84788.3799 - val_acc: 0.2442
Epoch 7/50
22667/22667 [==============================] - 2s - loss: 84267.2766 - acc: 0.2793 - val_loss: 84111.6902 - val_acc: 0.294

In [38]:
X_train = encoder2.predict(X_train)
X_test = encoder2.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 1024)
(4001, 1024)


In [39]:
autoencoder3, encoder3 = get_encoder(X_train, X_test, 512, 50)

Train on 22667 samples, validate on 4001 samples
Epoch 1/50
22667/22667 [==============================] - 1s - loss: 14006.0073 - acc: 0.0018 - val_loss: 13233.2534 - val_acc: 0.0030
Epoch 2/50
22667/22667 [==============================] - 1s - loss: 13351.1249 - acc: 0.0111 - val_loss: 12676.1877 - val_acc: 0.0637
Epoch 3/50
22667/22667 [==============================] - 1s - loss: 12740.6037 - acc: 0.2036 - val_loss: 12161.4466 - val_acc: 0.2989
Epoch 4/50
22667/22667 [==============================] - 1s - loss: 12301.2750 - acc: 0.3122 - val_loss: 11858.5385 - val_acc: 0.3752
Epoch 5/50
22667/22667 [==============================] - 1s - loss: 12041.9835 - acc: 0.3570 - val_loss: 11706.7873 - val_acc: 0.3962
Epoch 6/50
22667/22667 [==============================] - 1s - loss: 11866.5490 - acc: 0.3838 - val_loss: 11559.6472 - val_acc: 0.3799
Epoch 7/50
22667/22667 [==============================] - 1s - loss: 11744.8974 - acc: 0.4015 - val_loss: 11483.6837 - val_acc: 0.3939
Epoch 

In [40]:
X_train = encoder3.predict(X_train)
X_test = encoder3.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 512)
(4001, 512)
